SparkSession 생성

In [ ]:
from pyspark.sql import SparkSession
spark = (
    SparkSession.builder
    # 애플리케이션 이름: Spark UI에서 식별용
    .appName("MyFirstPipeline")

    # 실행 환경 설정
    # - "local[*]": 로컬 모드, 모든 코어 사용
    # - "spark://master:7077": 클러스터 모드
    .master("local[*]")

    # Executor 메모리: 각 워커가 사용할 메모리
    .config("spark.executor.memory", "2g")

    # 셔플 파티션 수: groupBy, join 등에서 사용
    # 기본값 200은 작은 데이터에 과도함 → 줄이면 성능 향상
    .config("spark.sql.shuffle.partitions", 10)

    # 세션 생성 또는 기존 세션 반환
    .getOrCreate()
)

print("설정된 SparkSession 생성 완료!")
print(f"  셔플 파티션: {spark.conf.get('spark.sql.shuffle.partitions')}")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/19 07:50:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/01/19 07:50:29 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


설정된 SparkSession 생성 완료!
  셔플 파티션: 10


26/01/19 07:50:41 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


Step 2: 데이터 읽기

In [2]:
from pyspark.sql.types import (
    StructType, StructField,
    StringType, IntegerType, DoubleType, BooleanType, DateType
)

employee_schema = StructType([
    StructField("emp_id",IntegerType(), False),
    StructField("name",StringType(), True),
    StructField("department",StringType(), True),
    StructField("salary",IntegerType(), True),
    StructField("hire_date",DateType(), True),
    StructField("age",IntegerType(), True),
    StructField("is_manager",BooleanType(), True)
])

df_with_schema = (
    spark.read
    .option("header", "true")
    .schema(employee_schema)                          # 스키마 지정
    .csv("/tmp/spark_tutorial/employees.csv")
)

print("=== 스키마 직접 지정 결과 ===")
df_with_schema.printSchema()

=== 스키마 직접 지정 결과 ===
root
 |-- emp_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- hire_date: date (nullable = true)
 |-- age: integer (nullable = true)
 |-- is_manager: boolean (nullable = true)



Step 3: 데이터 저장

In [4]:
df_with_schema.write \
    .mode("overwrite") \
    .partitionBy("department")\
    .parquet("/tmp/spark_tutorial/my_output")

print("Parquet 저장 완료!")

Parquet 저장 완료!


보너스: 파일 크기 비교

In [ ]:
# du -sh part-00000-32ef87b9-4f2a-485e-9a50-05022fb8b48e-c000.snappy.parquet
#4.0K    part-00000-32ef87b9-4f2a-485e-9a50-05022fb8b48e-c000.snappy.parquet
#du -sh employees.csv
#8.0K    employees.csv

In [ ]:
# Parquet 파일 읽기
df_parquet = spark.read.parquet("/tmp/spark_tutorial/my_output/department=Engineering")
df_parquet.show(4)

+------+-----------+------+----------+---+----------+
|emp_id|       name|salary| hire_date|age|is_manager|
+------+-----------+------+----------+---+----------+
|  NULL|Employee_24|  NULL|2020-06-10| 27|     false|
|  NULL|Employee_25|  NULL|2020-06-17| 31|     false|
|  NULL|Employee_34|  NULL|2020-08-19| 26|     false|
|  NULL|Employee_39|  NULL|2020-09-23| 29|     false|
+------+-----------+------+----------+---+----------+
only showing top 4 rows

